In [1]:
import mpbn

def bonesis_ensemble_from_single_bn(bn, limit=1000,
                    exact_influence_graph=True,
                    fixedpoints=None,
                    extra_properties=lambda bo: bo,
                    diversity=True,
                    maxclause=32,
                    **domain_opts):
    """
    bn: a colomoto.minibn.BooleanNetwork object or bnet filename
    limit: maximum size of the ensemble to generate
    exact_influence_graph: if True, all returned BNs have the exact same influence graph as `bn`. Otherwise, it can be a subgraph of it.
    fixedpoints:
        - None: no constraint
        - "included": all BNs include at least the fixed points of `bn`
        - "same": all BNs have exactly the same fixed points as `bn` (recommended to set `exact_influence_graph=False`)
    extra_properties: function that takes as input a BoNesis object and completes it with additional constraints.
    diversity: use diverse ensemble generation from bonesis
    """

    import bonesis

    bn = mpbn.MPBooleanNetwork.auto_cast(bn)
    dom = bonesis.InfluenceGraph(bn.influence_graph(),
                                 exact=exact_influence_graph,
                                 maxclause=maxclause, **domain_opts)


    data = {}
    if fixedpoints:
        for i, x in enumerate(bn.fixedpoints()):
            data[f"fp{i}"] = x

    bo = bonesis.BoNesis(dom, data)

    if fixedpoints in ["included", "same"]:
        for fp in data:
            bo.fixed(~bo.obs(fp))

    if fixedpoints == "same":
        bo.all_fixpoints({bo.obs(fp) for fp in data})

    extra_properties(bo)

    if diversity:
        view = bo.diverse_boolean_networks(limit=limit)
    else:
        view = bo.boolean_networks(limit=limit)

    return list(view)


In [3]:
bn = mpbn.MPBooleanNetwork('../models/synthetic_random_diff.bnet')

In [4]:
view = bonesis_ensemble_from_single_bn(bn, 50, fixedpoints="same", exact_influence_graph=False)

Grounding...done in 0.0s
Found 50 solutions in 1.6s (first in 0.0s; rate 0.0s)

In [5]:
if __name__ == "__main__":
    import sys
    bn = mpbn.MPBooleanNetwork(bn)
    print("input BN has", len(list(bn.fixedpoints())), "fixed points")

    print("-"*10, "no constraints:")
    for i, f in enumerate(bonesis_ensemble_from_single_bn(bn, 10)):
        fps = list(f.fixedpoints())
        print("bn", i, "has", len(fps), "fixed points")
    print()

    print("fixedpoints=included")
    for i, f in enumerate(bonesis_ensemble_from_single_bn(bn, 10, fixedpoints="included")):
        fps = list(f.fixedpoints())
        print("bn", i, "has", len(fps), "fixed points")
    print()

    print("fixedpoints=same", "exact_influence_graph=False")
    for i, f in enumerate(bonesis_ensemble_from_single_bn(bn, 10, fixedpoints="same",
                                                   exact_influence_graph=False)):
        fps = list(f.fixedpoints())
        print("bn", i, "has", len(fps), "fixed points")
    print()


input BN has 3 fixed points
---------- no constraints:
Grounding...done in 0.0s
Found 10 solutions in 0.2s (first in 0.0s; rate 0.0s)bn 0 has 1 fixed points
bn 1 has 1 fixed points
bn 2 has 2 fixed points
bn 3 has 1 fixed points
bn 4 has 1 fixed points
bn 5 has 2 fixed points
bn 6 has 1 fixed points
bn 7 has 1 fixed points
bn 8 has 1 fixed points
bn 9 has 1 fixed points

fixedpoints=included
Grounding...done in 0.0s
Found 10 solutions in 0.3s (first in 0.0s; rate 0.0s)bn 0 has 3 fixed points
bn 1 has 4 fixed points
bn 2 has 3 fixed points
bn 3 has 3 fixed points
bn 4 has 3 fixed points
bn 5 has 3 fixed points
bn 6 has 3 fixed points
bn 7 has 4 fixed points
bn 8 has 3 fixed points
bn 9 has 3 fixed points

fixedpoints=same exact_influence_graph=False
Grounding...done in 0.0s
Found 10 solutions in 0.2s (first in 0.0s; rate 0.0s)bn 0 has 3 fixed points
bn 1 has 3 fixed points
bn 2 has 3 fixed points
bn 3 has 3 fixed points
bn 4 has 3 fixed points
bn 5 has 3 fixed points
bn 6 has 3 fixed po

In [14]:
bn.influence_graph()

# computing graph layout...
